In [2]:
from pathlib import Path
import pandas as pd
import re

In [3]:
def export_onnx(path, output_path, opset):
    import pickle
    import warnings

    import torch
    from nnsmith.materialize import Model, Oracle
    from torch.onnx.verification import find_mismatch

    warnings.simplefilter("ignore")

    # Get the paths for pickles and weights
    gir_path: Path = path / "gir.pkl"
    oracle_path: Path = path / "oracle.pkl"
    weights_path: Path = path / "model.pth"

    # Load the model from pickle
    with gir_path.open("rb") as f:
        gir = pickle.load(f)
    model_type = Model.init("torch", "cpu")
    model = model_type.from_gir(gir)

    # Load weights from weight path.
    model.torch_model.load_state_dict(torch.load(weights_path), strict=False)

    # Load oracle
    oracle = Oracle.load(oracle_path)

    model_args = tuple([torch.from_numpy(val) for key, val in oracle.input.items()])

    print(f"Testing: {str(path)}")
    torch.onnx.export(
        model.torch_model, model_args, output_path, opset_version=opset
    )

In [4]:
def mismatch_test(path, opset):
    import pickle
    import warnings

    import torch
    from nnsmith.materialize import Model, Oracle
    from torch.onnx.verification import find_mismatch

    warnings.simplefilter("ignore")

    # Get the paths for pickles and weights
    gir_path: Path = path / "gir.pkl"
    oracle_path: Path = path / "oracle.pkl"
    weights_path: Path = path / "model.pth"

    # Load the model from pickle
    with gir_path.open("rb") as f:
        gir = pickle.load(f)
    model_type = Model.init("torch", "cpu")
    model = model_type.from_gir(gir)

    # Load weights from weight path.
    model.torch_model.load_state_dict(torch.load(weights_path), strict=False)

    # Load oracle
    oracle = Oracle.load(oracle_path)

    model_args = tuple([torch.from_numpy(val) for key, val in oracle.input.items()])

    print(f"Testing: {str(path)}")
    graph_info = find_mismatch(model.torch_model, model_args, opset_version=opset)
    return graph_info

In [5]:
def create_mismatch_test_dir(method, framework, opset, model_name, output_dir):
    mismismatch_test_path = Path(f"./{output_dir}/{framework}/{method}/{opset}/{model_name}")
    if not mismismatch_test_path.exists():
        mismismatch_test_path.mkdir(parents=True)
    return mismismatch_test_path

# Opset 16 -- excluded

In [10]:
results_path = Path("./exporter_test_results/torch")

In [11]:
dataframes = []
for path in  sorted(results_path.glob(f"torch_symbolic-cinit_opset_16_*.json")):
    dataframes.append(pd.read_json(path))

In [12]:
all_data_opset16 = pd.concat(dataframes).fillna(0)

In [13]:
all_data_opset16

,model,path,result,error,mismatch
0,model_0_242079451,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
1,model_10_806380245,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
2,model_11_1158151431,nnsmith_constrained/symbolic-cinit/torch/100/m...,1,[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Cou...,0.0
3,model_12_803349300,nnsmith_constrained/symbolic-cinit/torch/100/m...,1,[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Cou...,0.0
4,model_13_721245967,nnsmith_constrained/symbolic-cinit/torch/100/m...,1,[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Cou...,0.0
...,...,...,...,...,...
94,model_96_2247477139,nnsmith_constrained/symbolic-cinit/torch/95/mo...,1,[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Cou...,0.0
95,model_97_1071676939,nnsmith_constrained/symbolic-cinit/torch/95/mo...,0,0,0.0
96,model_98_78593957,nnsmith_constrained/symbolic-cinit/torch/95/mo...,1,[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Cou...,0.0
97,model_99_2255557066,nnsmith_constrained/symbolic-cinit/torch/95/mo...,1,[ONNXRuntimeError] : 10 : INVALID_GRAPH : This...,0.0


In [14]:
all_data_opset16[['result', 'mismatch']].value_counts()

result  mismatch
0       0.0         881
1       0.0         743
0       1.0         100
Name: count, dtype: int64

In [15]:
all_data_opset16.loc[(all_data_opset16["mismatch"] == 0) & (all_data_opset16["result"] == 0)]

,model,path,result,error,mismatch
0,model_0_242079451,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
1,model_10_806380245,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
16,model_24_1364982628,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
18,model_26_2756794408,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
24,model_31_3918646064,nnsmith_constrained/symbolic-cinit/torch/100/m...,0,0,0.0
...,...,...,...,...,...
88,model_8_1517613889,nnsmith_constrained/symbolic-cinit/torch/95/mo...,0,0,0.0
89,model_90_304262387,nnsmith_constrained/symbolic-cinit/torch/95/mo...,0,0,0.0
91,model_92_2947853082,nnsmith_constrained/symbolic-cinit/torch/95/mo...,0,0,0.0
92,model_93_3157390059,nnsmith_constrained/symbolic-cinit/torch/95/mo...,0,0,0.0


In [13]:
x = all_data_opset16.loc[(all_data_opset16["result"] == 1)][["model", 'error', 'path']]

for idx, row in x.iterrows():
    if "ONNXRun" in row['error']:
        output_path = "failing_model/torch/" + row['model'] + "_const_initializers.onnx"
        print("----------")
        print(row['model'])
        print(row['error'])
        print(row['path'])
        # export_onnx(Path(row['path']), output_path, 16)

----------
model_11_1158151431
[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for ArgMax(13) node with name 'ArgMax_211'
nnsmith_constrained/symbolic-cinit/torch/100/model_11_1158151431
----------
model_12_803349300
[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Cos(7) node with name 'Cos_194'
nnsmith_constrained/symbolic-cinit/torch/100/model_12_803349300
----------
model_13_721245967
[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Tan(7) node with name 'Tan_169'
nnsmith_constrained/symbolic-cinit/torch/100/model_13_721245967
----------
model_14_2408570878
[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Resize(13) node with name 'Resize_170'
nnsmith_constrained/symbolic-cinit/torch/100/model_14_2408570878
----------
model_16_2778272357
[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Atan(7) node with name 'Atan_207'
nnsmith_const

In [ ]:
mismatched_paths = [Path(i) for i in all_data_opset16.loc[all_data_opset16["mismatch"] == 1]['path']]

In [ ]:
additional_test = {}
for mismatched_path in mismatched_paths:
    model_subgraph_directory = create_mismatch_test_dir('symbolic-cinit', 'torch', 16, mismatched_path.name, "mismatched_subgraphs")
    graph_info = mismatch_test(mismatched_path, 16)
    mismatched_nodes = graph_info.all_mismatch_leaf_graph_info()

    for i, mismatched_node in enumerate(mismatched_nodes):
        mismatched_subgraph = model_subgraph_directory / f"{i}"
        mismatched_node.export_repro(mismatched_subgraph)

In [ ]:
num_ort = []
num_not_ort = []
n = 0
for idx, row in all_data_opset16.loc[all_data_opset16['result'] == 1].iterrows():
    if "ONNXRuntimeError" in row['error']:
        num_ort.append((row['model'], row['error']))
    else:
        num_not_ort.append((row['model'], row['error']))
    n += 1

In [ ]:
len(num_ort)

In [ ]:
num_not_ort

In [ ]:
len(num_not_ort)

# Find Ops to Exclude

## Opset 16

In [ ]:
opset16_errors = set(all_data_opset16.loc[all_data_opset16['result'] == 1]['error'])

In [ ]:
len(opset16_errors)

In [ ]:
opset16_errors

In [ ]:
not_imp_opset16_errors = list(filter(lambda x: "NOT_IMPLEMENTED" in x, opset16_errors))

In [ ]:
len(not_imp_opset16_errors)

In [ ]:
pattern = r"for\s([^\s(]+)\s*\([^)]*\)\s*node"
erring_nodes = set(
    [
        re.search(pattern, err).group(1) if re.search(pattern, err) else err
        for err in not_imp_opset16_errors
    ]
)
erring_nodes


In [ ]:
other_opset16_errors = list(filter(lambda x: not "NOT_IMPLEMENTED" in x, opset16_errors))

In [ ]:
other_opset16_errors

In [ ]:
all_data_opset16.loc[all_data_opset16['error'] ==  '[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Failed to find kernel for FusedMatMul(1) (node MatMul_With_Transpose). Op with name (MatMul_With_Transpose) domain (com.microsoft) and type (FusedMatMul) kernel is not supported in CPUExecutionProvider. Encountered following errors: (Kernel found kernel in the supported version range (node_version: 1). However the types are incompatible. This op has been implemented only for the following types (tensor(float),), but the node in the model has the following type (tensor(double))\n)']

In [ ]:
print(
    all_data_opset16.loc[
        all_data_opset16["error"]
        == "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Failed to find kernel for FusedMatMul(1) (node MatMul_With_Transpose). Op with name (MatMul_With_Transpose) domain (com.microsoft) and type (FusedMatMul) kernel is not supported in CPUExecutionProvider. Encountered following errors: (Kernel found kernel in the supported version range (node_version: 1). However the types are incompatible. This op has been implemented only for the following types (tensor(float),), but the node in the model has the following type (tensor(double))\n)"
    ]["path"].item()
)


In [ ]:
export_onnx(Path("nnsmith_constrained/symbolic-cinit/torch/90/model_81_4287267281"), 16)

# Log Data

In [ ]:
log_data = pd.read_json('log_data.json')

In [ ]:
all_diff_data = log_data.loc[~log_data['greatest abs diff.'].isna()]

In [ ]:
all_diff_data.sort_values('greatest abs diff.', ascending=False)
all_diff_data['model'] = all_diff_data['path'].str.strip('.log')

In [ ]:
x = all_diff_data.sort_values('greatest abs diff.', ascending=False).head(15)[['model', 'greatest abs diff.']]

In [ ]:
top_mismatched = all_data_opset16.loc[all_data_opset16['mismatch'] == 1].set_index('model').join(x.set_index('model')).sort_values('greatest abs diff.', ascending=False)

In [ ]:
top_mismatched.iloc[0,:]['path']

In [106]:
top_mismatched[['greatest abs diff.']].to_markdown('./md.md')

In [115]:
for path in top_mismatched['path']:
    print(path)

nnsmith_constrained/symbolic-cinit/torch/40/model_78_3459169476
nnsmith_constrained/symbolic-cinit/torch/90/model_68_72227799
nnsmith_constrained/symbolic-cinit/torch/50/model_6_419668759
nnsmith_constrained/symbolic-cinit/torch/20/model_96_728966504
nnsmith_constrained/symbolic-cinit/torch/100/model_30_213242519
nnsmith_constrained/symbolic-cinit/torch/100/model_15_2262524614
nnsmith_constrained/symbolic-cinit/torch/80/model_29_103575771
nnsmith_constrained/symbolic-cinit/torch/75/model_78_2707514266
nnsmith_constrained/symbolic-cinit/torch/35/model_60_674754574
nnsmith_constrained/symbolic-cinit/torch/90/model_91_3302636491
nnsmith_constrained/symbolic-cinit/torch/75/model_50_1243263724
nnsmith_constrained/symbolic-cinit/torch/30/model_74_3620935663
nnsmith_constrained/symbolic-cinit/torch/100/model_6_3867095439
nnsmith_constrained/symbolic-cinit/torch/40/model_25_1123596368
nnsmith_constrained/symbolic-cinit/torch/40/model_23_3959668477


In [120]:
import numpy as np
import onnxruntime as ort
import pickle
import warnings
import torch
import shutil
from nnsmith.materialize import Model, Oracle

warnings.simplefilter("ignore")

for path in top_mismatched['path']:
    path = Path(path)

    # Get the paths for pickles and weights
    shutil.copytree(path, f"./bugs/torch/bug_2/{path.name}")

    # gir_path: Path = path / "gir.pkl"
    # oracle_path: Path = path / "oracle.pkl"
    # weights_path: Path = path / "model.pth"

    # # Load the model from pickle
    # with gir_path.open("rb") as f:
    #     gir = pickle.load(f)
    # model_type = Model.init("torch", "cpu")
    # model = model_type.from_gir(gir)

    # # Load weights from weight path.
    # model.torch_model.load_state_dict(torch.load(weights_path), strict=False)

    # # Load oracle
    # oracle = Oracle.load(oracle_path)

    # model_args = tuple([torch.from_numpy(val) for key, val in oracle.input.items()])

    # print(f"Testing: {str(path)}")
    # traced_model = torch.jit.trace(model.torch_model, model_args)
    # traced_model.save(f"./bugs/torch/bug_2/{str(path.name)}.pt")
    # shutil.copy(weights_path, f"./bugs/torch/bug_2/{str(path.name)}_weights.pt")

    # torch.save(oracle.input, f"./bugs/torch/bug_2/{str(path.name)}_inputs.pt")

    # inps = [torch.Tensor(oracle.input[i]) for i in oracle.input]
    # torch.onnx.export(traced_model, model_args, f"./bugs/torch/bug_2/{str(path.name)}.onnx", opset_version=16, input_names=list(model.input_like), output_names=list(model.output_like))

    # traced_output = traced_model(*model_args)
    # sess = ort.InferenceSession(f"./bugs/torch/bug_2/{str(path.name)}.onnx", providers=["CPUExecutionProvider"])
    # out_1 = sess.run(list(model.output_like), oracle.input)

    # maxes = []
    # for i,j in zip(out_1, traced_output):
    #     j = j.detach().numpy()
    #     try:
    #         out = i - j
    #     except:
    #         out = i ^ j
    #     maxes.append(out.max())
    # print(np.array(maxes).max())

# Path Analysis

## Mismatched Path comparison

In [ ]:
import json
from difflib import SequenceMatcher
from pathlib import Path
import pandas as pd

In [ ]:
results_path = Path("./exporter_test_results_new/torch")

In [ ]:
dataframes = []
for path in  sorted(results_path.glob(f"torch_symbolic-cinit_opset_16_*.json")):
    dataframes.append(pd.read_json(path))

In [ ]:
all_data_opset16 = pd.concat(dataframes).fillna(0)

In [ ]:
all_data_opset16.loc[all_data_opset16['mismatch'] == 1]

In [ ]:
with open('mismatch_seq_match_results.json', 'r') as f:
    mismatched = json.load(f)

In [ ]:
token_node_type_dict = mismatched['token_node_type_dict']

In [ ]:
total_sequences = 0
mismatched_sequences = set()
for i, (_, sequences) in enumerate(mismatched['results'].items()):
    total_sequences += len(sequences)
    mismatched_sequences |= set(sequences)

mismatched_sequences_decoded = []
for sequence in mismatched_sequences:
    decoded_sequence = [token_node_type_dict[i] for i in sequence]
    mismatched_sequences_decoded.append(decoded_sequence)

print(f"Total Sequences: {total_sequences}, Total Unique Sequences: {len(mismatched_sequences)}")

## Mismatched-Correct Path comparison

In [ ]:
with open('correct_mismatch_seq_match_results.json', 'r') as f:
    mismatched_correct = json.load(f)

In [ ]:
token_node_type_dict = mismatched_correct['token_node_type_dict']

In [ ]:
mismatched_correct_sequences = set()
total_sequences = 0
for i, (_, sequences) in enumerate(mismatched_correct['results'].items()):
    total_sequences += len(sequences)
    mismatched_correct_sequences |= set(sequences)

mismatched_correct_sequences_decoded = []
for sequence in mismatched_correct_sequences:
    decoded_sequence = [token_node_type_dict[i] for i in sequence]
    mismatched_correct_sequences_decoded.append(decoded_sequence)

print(f"Total Sequences: {total_sequences}, Total Unique Sequences: {len(mismatched_correct_sequences_decoded)}")

In [ ]:
len(mismatched_sequences - mismatched_correct_sequences)

In [ ]:
all_non_overlapping_sequences = mismatched_sequences - mismatched_correct_sequences

In [ ]:
all_non_overlapping_sequences_decoded = [[token_node_type_dict[i] for i in sequence] for sequence in all_non_overlapping_sequences]

In [ ]:
all_non_overlapping_sequences_decoded

In [ ]:
import json

In [ ]:
with open("onnx_parsing_results/torch_test_mismatch_seq_match_results.json", 'r') as f:
    mismatched_correct = json.load(f)

In [ ]:
mismatched_correct_sequences = set()
total_sequences = 0
for i, (_, sequences) in enumerate(mismatched_correct['results'].items()):
    total_sequences += len(sequences)
    mismatched_correct_sequences |= set(sequences)

In [ ]:
mismatched_correct['total_path_pairs_analyzed']

In [ ]:
len(mismatched_correct_sequences)

In [ ]:
total_sequences